In [19]:
from kafka import KafkaProducer
from kafka.errors import KafkaError
import msgpack
import json

# KafkaProducer

In [26]:
# create a KafkaProducer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

# 1. Asynchronous by default
basic sending, it works as well. just for prototyping

In [17]:

future = producer.send('quick-events', b'sensor_from python')


In [5]:
future

In [7]:
# Block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
except KafkaError:
    # Decide what to do if produce request failed...
    log.exception()
    pass

In [8]:
# Successful result returns assigned partition and offset
print (record_metadata.topic)
print (record_metadata.partition)
print (record_metadata.offset)

quick-events
0
9


# 2. Produce keyed messages to enable hashed partitioning

In [16]:

producer.send('quick-events', key=b'foo', value=b'bar') # other way to send

# 3. Encode objects via msgpack
it is not showing as well in the kafka consumer. I think it is not decoded entirely

In [15]:

producer = KafkaProducer(value_serializer=msgpack.dumps)
producer.send('quick-events', {'key': 'value'})

# 4. Produce json messages 
it worrked as well on kafka consumer

In [20]:

producer = KafkaProducer(value_serializer=lambda m: json.dumps(m).encode('ascii'))
producer.send('quick-events', {'key': 'value'})

In [21]:

def on_send_success(record_metadata):
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)
    # handle exception


In [28]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'], value_serializer=lambda m: json.dumps(m).encode('ascii'))
producer.send('quick-events', {'sensor_2': 25}).add_callback(on_send_success).add_errback(on_send_error)


quick-events
0
19


In [29]:
producer.bootstrap_connected()

False